# Preliminary distributions

In [1]:
USE TmsEPrd;

Commands completed successfully.

Total execution time: 00:00:00.001

## Save prelims into temp table

In [2]:
SELECT
    d.PRELIM_ARCH_VERSION_ID,
    d.YR_CDE,
    d.TRM_CDE,
    d.CRS_CDE_HIER,
    d.LOC_CDE,
    LEFT(d.ORIGINAL_ACCT_CDE, 15) FROM_DEPT,
    LEFT(d.REVENUE_ACCT_CDE, 15) TO_DEPT,
    CONCAT(LEFT(CONVERT(VARCHAR(8), d.ID_NUM), 1), 'xxxxx') ID,
    d.CHG_FEE_CDE,
    -d.TRANS_AMT TRANS_AMT,
    d.STUD_GEN_INTENT,
    d.METHOD
INTO #jjj_prelim
FROM TWU_PRELIM_ARCH_VERSION h
JOIN TWU_PRELIM_ARCH d ON h.PRELIM_ARCH_VERSION_ID = d.PRELIM_ARCH_VERSION_ID
WHERE h.LAST_MODIFIED_TIME > '2019-08-01'
AND d.TRANS_AMT < 0;

(611116 rows affected)

Total execution time: 00:00:02.617

## Create results temp table

In [3]:
CREATE TABLE #jjj_results (
    [ResultId] INT IDENTITY(1,1),
    [PrelimId] INT,
    [Metric] VARCHAR(16),
    [Key] VARCHAR(128),
    [Value] NUMERIC(15,2)
);

Commands completed successfully.

Total execution time: 00:00:00.004

## Create helper procedure

In [4]:
CREATE PROCEDURE jjj_measure
    @MetricName VARCHAR(16),
    @KeyField VARCHAR(1024),
    @ValueField VARCHAR(1024)
AS
BEGIN
    DECLARE @script NVARCHAR(MAX) = REPLACE(REPLACE(REPLACE(
        'INSERT INTO #jjj_results ([PrelimId], [Metric], [Key], [Value])
        SELECT
            PRELIM_ARCH_VERSION_ID,
            ''{MetricName}'',
            COALESCE({KeyField}, ''Unknown''),
            {ValueField}
        FROM #jjj_prelim
        GROUP BY PRELIM_ARCH_VERSION_ID, {KeyField}',
        '{ValueField}', @ValueField),
        '{KeyField}', @KeyField),
        '{MetricName}', @MetricName);
    PRINT @script;
    EXEC sp_executesql @script;
END

Commands completed successfully.

Total execution time: 00:00:00.006

## Measure using different metrics

In [5]:
EXEC jjj_measure 'YearTerm', 'CONCAT(YR_CDE, TRM_CDE)', 'SUM(TRANS_AMT)';
--EXEC jjj_measure 'Course', 'CRS_CDE_HIER', 'SUM(TRANS_AMT)';
EXEC jjj_measure 'Location', 'LOC_CDE', 'SUM(TRANS_AMT)';
EXEC jjj_measure 'FromDept', 'FROM_DEPT', 'SUM(TRANS_AMT)';
EXEC jjj_measure 'ToDept', 'TO_DEPT', 'SUM(TRANS_AMT)';
EXEC jjj_measure 'ID', 'ID', 'SUM(TRANS_AMT)';
EXEC jjj_measure 'FeeCode', 'CHG_FEE_CDE', 'SUM(TRANS_AMT)';
EXEC jjj_measure 'Intent', 'STUD_GEN_INTENT', 'SUM(TRANS_AMT)';
EXEC jjj_measure 'Method', 'METHOD', 'SUM(TRANS_AMT)';

INSERT INTO #jjj_results ([PrelimId], [Metric], [Key], [Value])
 SELECT
 PRELIM_ARCH_VERSION_ID,
 'YearTerm',
 COALESCE(CONCAT(YR_CDE, TRM_CDE), 'Unknown'),
 SUM(TRANS_AMT)
 FROM #jjj_prelim
 GROUP BY PRELIM_ARCH_VERSION_ID, CONCAT(YR_CDE, TRM_CDE)

(34 rows affected)

INSERT INTO #jjj_results ([PrelimId], [Metric], [Key], [Value])
 SELECT
 PRELIM_ARCH_VERSION_ID,
 'Location',
 COALESCE(LOC_CDE, 'Unknown'),
 SUM(TRANS_AMT)
 FROM #jjj_prelim
 GROUP BY PRELIM_ARCH_VERSION_ID, LOC_CDE

(140 rows affected)

INSERT INTO #jjj_results ([PrelimId], [Metric], [Key], [Value])
 SELECT
 PRELIM_ARCH_VERSION_ID,
 'FromDept',
 COALESCE(FROM_DEPT, 'Unknown'),
 SUM(TRANS_AMT)
 FROM #jjj_prelim
 GROUP BY PRELIM_ARCH_VERSION_ID, FROM_DEPT

...

INSERT INTO #jjj_results ([PrelimId], [Metric], [Key], [Value])
 SELECT
 PRELIM_ARCH_VERSION_ID,
 'Intent',
 COALESCE(STUD_GEN_INTENT, 'Unknown'),
 SUM(TRANS_AMT)
 FROM #jjj_prelim
 GROUP BY PRELIM_ARCH_VERSION_ID, STUD_GEN_INTENT

(688 rows affected)

INSERT INTO #jjj_results ([PrelimId], [Metric], [Key], [Value])
 SELECT
 PRELIM_ARCH_VERSION_ID,
 'Method',
 COALESCE(METHOD, 'Unknown'),
 SUM(TRANS_AMT)
 FROM #jjj_prelim
 GROUP BY PRELIM_ARCH_VERSION_ID, METHOD

(57 rows affected)

Total execution time: 00:00:05.910

## Measure count and sum totals

In [5]:
INSERT INTO #jjj_results ([PrelimId], [Metric], [Key], [Value])
SELECT
    PRELIM_ARCH_VERSION_ID,
    'Count' Metric,
    'Total' [Key],
    COUNT(*) [Value]
FROM #jjj_prelim
GROUP BY PRELIM_ARCH_VERSION_ID;

INSERT INTO #jjj_results ([PrelimId], [Metric], [Key], [Value])
SELECT
    PRELIM_ARCH_VERSION_ID,
    'Sum' Metric,
    'Total' [Key],
    SUM(TRANS_AMT) [Value]
FROM #jjj_prelim
GROUP BY PRELIM_ARCH_VERSION_ID;


(19 rows affected)

(19 rows affected)

Total execution time: 00:00:00.536

## Show results

In [6]:
SELECT *
FROM #jjj_results;

(2909 rows affected)

Total execution time: 00:00:00.157

ResultId,PrelimId,Metric,Key,Value
175,54,FromDept,1 10 00400,1015848.00
176,57,FromDept,1 10 00400,1036646.00
177,60,FromDept,1 10 00400,1019508.00
178,63,FromDept,1 10 00400,1022476.00
179,70,FromDept,1 10 00400,1064344.00
180,58,FromDept,1 10 06000,198629.50
181,61,FromDept,1 10 06000,201181.00
182,64,FromDept,1 10 06000,195455.00
183,67,FromDept,1 10 06000,202196.00
184,53,FromDept,1 10 20100,28295279.00


## Clean up

In [8]:
DROP PROCEDURE jjj_measure;
DROP TABLE #jjj_results;
DROP TABLE #jjj_prelim;

: Msg 3701, Level 11, State 5, Line 2
Cannot drop the table '#jjj_results', because it does not exist or you do not have permission.

: Msg 3701, Level 11, State 5, Line 3
Cannot drop the table '#jjj_prelim', because it does not exist or you do not have permission.

Total execution time: 00:00:00.109